<a href="https://colab.research.google.com/github/cookkiiee/last_project/blob/main/%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%84%9C%EC%9A%B8%EA%B8%B0%EC%83%81%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# github에서 서울 기상 데이터세트를 다운받습니다.
!git clone https://github.com/baek2sm/ml.git

fatal: destination path 'ml' already exists and is not an empty directory.


In [15]:
# 실습에 필요한 라이브러리를 불러옵니다.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib

In [16]:
# 서울 기상 데이터세트를 불러옵니다.
datasets = joblib.load('./ml/datasets/weather.pickle')

# 입력 데이터와 타깃을 준비합니다.
data, target = datasets['data'], datasets['target']

# 입력 데이터와 타깃의 형태를 출력합니다.
print(data.shape, target.shape)

(26181, 24, 3) (26181,)


In [17]:
# 학습 세트와 테스트 세트를 나눕니다.
train_length = 20000
X_train, X_test = data[:train_length], data[train_length:]
y_train, y_test = target[:train_length], target[train_length:]

# 넘파이 자료구조를 텐서 자료구조로 변환합니다.
X_train, X_test = torch.from_numpy(X_train), torch.from_numpy(X_test)
y_train, y_test = torch.from_numpy(y_train), torch.from_numpy(y_test)

In [18]:
# 데이터세트 자료구조를 준비합니다.
dset_train, dset_test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)

# 데이터 로더를 준비합니다.
train_loader = DataLoader(dset_train, batch_size=256, shuffle=True)
test_loader = DataLoader(dset_test, batch_size=256, shuffle=False)

In [19]:
# RNN 모델 클래스를 정의합니다.
class RNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.cell = nn.RNN(3, 3, batch_first=True)
    self.fc = nn.Linear(24*3, 1)

  def forward(self, X):
    out, hidden_state = self.cell(X)
    out = out.contiguous()
    out = self.fc(out.view(-1, 24*3))
    return out

In [20]:
# 그래픽 카드 사용이 가능할 경우 그래픽 카드로 연산하도록 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# RNN 모델 객체를 생성합니다.
model = RNN().to(device)

# 평균 제곱 오차(Mean Squared Error) 손실 함수 객체를 생성합니다.
criterion = nn.MSELoss()

# 아담 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=2e-4)

In [21]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차를 저장할 변수를 생성합니다.
  epoch_loss = 0

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:
    # 입력 데이터와 타깃을 준비합니다.
    X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 예측합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()    
    # 현재 배치의 오차를 저장합니다.
    epoch_loss += loss.item()

  # 현재 에포크의 오차를 반환합니다.
  return epoch_loss / len(loader)

In [22]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, loader):
  # 현재 에포크의 오차를 저장할 변수를 생성합니다.
  epoch_loss = 0

  # 모델을 평가 모드로 설정합니다.
  model.eval()

  with torch.no_grad():
    # 배치 학습을 실행합니다.
    for X_batch, y_batch in loader:
      # 입력 데이터와 타깃을 그래픽카드로 연산하도록 준비합니다.
      X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
      # 모델을 사용해 타깃을 예측합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 현재 배치의 오차를 저장합니다.
      epoch_loss += loss.item()

    # 현재 에포크의 오차를 반환합니다.
    return epoch_loss / len(loader)

In [13]:
# 200회에 걸쳐 모델을 학습시킵니다.
for epoch in range(1, 201):
  # 모델을 학습시킵니다.
  loss = train(model, criterion, optimizer, train_loader)

  # 모델을 평가합니다.
  test_loss = evaluate(model, criterion, test_loader)

  if epoch % 20 == 0:
    # 20 에포크마다 현재 에포크의 학습 결과를 출력합니다.
    print('epoch: {}, loss: {:.3f}, test_loss: {:.3f}'.format(
        epoch, loss, test_loss
    )) 

epoch: 20, loss: 47.024, test_loss: 29.086
epoch: 40, loss: 20.240, test_loss: 19.488
epoch: 60, loss: 16.596, test_loss: 17.943
epoch: 80, loss: 14.374, test_loss: 15.693
epoch: 100, loss: 12.703, test_loss: 14.161
epoch: 120, loss: 11.753, test_loss: 13.363
epoch: 140, loss: 11.233, test_loss: 12.848
epoch: 160, loss: 10.754, test_loss: 12.437
epoch: 180, loss: 10.368, test_loss: 12.158
epoch: 200, loss: 10.198, test_loss: 11.891


In [23]:
# 모델을 평가 모드로 설정합니다.
model.eval()

with torch.no_grad():    
    # 테스트 세트의 데이터로더에서 배치 단위로 데이터를 불러옵니다.
    for X_batch, y_batch in test_loader:
      # 배치 데이터를 cuda로 이동시키고, 타깃의 타입과 형태를 조정합니다.
      X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
      # 모델을 사용해 타깃을 예측합니다. 예측값은 10개 클래스에 대한 확률값입니다.
      hypothesis = model(X_batch)      
      # 배치의 첫 번째 데이터 샘플마다 모델이 예측한 값과 실제 타깃을 출력합니다.
      print('predicted target: {:.2f}, real target: {:.2f}'.format(hypothesis[0].item(), y_batch[0].item()))

predicted target: 0.25, real target: 18.40
predicted target: 0.55, real target: 5.20
predicted target: 0.23, real target: 11.50
predicted target: 0.27, real target: 8.90
predicted target: 0.27, real target: -0.20
predicted target: 0.27, real target: 0.20
predicted target: 0.27, real target: 0.30
predicted target: 0.27, real target: 5.00
predicted target: 0.27, real target: -1.20
predicted target: 0.27, real target: 3.60
predicted target: 0.27, real target: 2.70
predicted target: 0.27, real target: 5.20
predicted target: 0.27, real target: 8.20
predicted target: 0.27, real target: 0.50
predicted target: -0.02, real target: 3.60
predicted target: 0.54, real target: 16.00
predicted target: 0.23, real target: 7.30
predicted target: -0.23, real target: 13.20
predicted target: 0.33, real target: 15.80
predicted target: 0.28, real target: 11.70
predicted target: 0.39, real target: 17.70
predicted target: 0.27, real target: 21.60
predicted target: 0.18, real target: 19.30
predicted target: -0.